In [1]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
import datetime
import geopy.distance
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'
storage = "/Volumes/easystore/Drones"
coi=['Incident No.','Description','cfs-date','Block Location','Latitude','Longitude','drone-id']

calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

flight_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_drone-seconds-block-count-per-flight.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/mapping/cv-blocks.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks['GEOID20'] = cv_blocks['GEOID20'].apply(lambda x: x[3:])
cv_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../data/outputs/outputs_cv-flight-manifest.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [29]:
cv_flights[cv_flights['type'].astype(str).str.contains('noise',case=False)]

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,drone-date
6921,32.7126771,-117.1628885,10:35pm,fc555966795665f36adaf899ae46edb0,2023-04-02,10:35pm,200 G St,CVL027640,Noise Complaint,cvpd,2023-04-02
8548,32.6272023,-117.0618539,7:39pm,bb7d42abac90b498ceac2cf15b642458,2023-07-09,7:39pm,00 K St,L057992,Noise Complaint,cvpd,2023-07-09


In [27]:
cv_flights[cv_flights['id']=='fc555966795665f36adaf899ae46edb0']

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,drone-date
6921,32.7126771,-117.1628885,10:35pm,fc555966795665f36adaf899ae46edb0,2023-04-02,10:35pm,200 G St,CVL027640,Noise Complaint,cvpd,2023-04-02


In [2]:
#060730131032011 <- harborside
#060730125023005 <- circlek

In [30]:
calls_for_service["cfs-date"] = pd.to_datetime(calls_for_service["Date"])
cv_flights["drone-date"] = pd.to_datetime(cv_flights["date"])


In [4]:
circle_k = flight_blocks[(flight_blocks['GEOID20']=='060730124011022')|(flight_blocks['GEOID20']=='060730124011021')|
(flight_blocks['GEOID20']=='060730124011019')]
circle_k = circle_k[circle_k['seconds'].astype(int)>60]
circle_k_flight_ids = circle_k['id'].drop_duplicates().values
circle_k_flights = cv_flights[cv_flights['id'].isin(circle_k_flight_ids)]
circle_k_flights = circle_k_flights[circle_k_flights["drone-date"]>"2023-08-01"]
circle_k_flights[circle_k_flights['type'].astype(str).str.contains('domestic')].values

array([['32.63997389', '-117.08271497', '5:13pm',
        '5dead61fec3d34d46f0428c7dc72ba0c', '2023-08-09', '5:13pm',
        '45 north broadway Avenue', 'Cvl67832',
        'domestic dispute two subjects seen at the bus stop together no dv seen',
        'cvpd', Timestamp('2023-08-09 00:00:00')]], dtype=object)

In [23]:
circle_k_flights[circle_k_flights['id']=='5dead61fec3d34d46f0428c7dc72ba0c']

array([['32.63997389', '-117.08271497', '5:13pm',
        '5dead61fec3d34d46f0428c7dc72ba0c', '2023-08-09', '5:13pm',
        '45 north broadway Avenue', 'Cvl67832',
        'domestic dispute two subjects seen at the bus stop together no dv seen',
        'cvpd', Timestamp('2023-08-09 00:00:00')]], dtype=object)

In [33]:
calls_for_service.columns
# cv_flights[cv_flights['id']=='fc555966795665f36adaf899ae46edb0']

Index(['Incident No.', 'Date', 'Description', 'Priority', 'Disposition',
       'Call Source', 'Zip Code', 'Block Location', 'Case No', 'Address',
       'Latitude', 'Longitude', 'Accuracy Score', 'Accuracy Type', 'Number',
       'Street', 'Unit Type', 'Unit Number', 'City', 'State', 'County', 'Zip',
       'Country', 'Source', 'Census Year', 'State FIPS', 'County FIPS',
       'Place Name', 'Place FIPS', 'Census Tract Code', 'Census Block Code',
       'Census Block Group', 'Full FIPS (block)', 'Full FIPS (tract)',
       'Metro/Micro Statistical Area Name',
       'Metro/Micro Statistical Area Code',
       'Metro/Micro Statistical Area Type', 'Combined Statistical Area Name',
       'Combined Statistical Area Code', 'Metropolitan Division Area Name',
       'Metropolitan Division Area Code', 'cfs-date'],
      dtype='object')

In [35]:
calls_for_service[((calls_for_service['cfs-date'])>"2023-04-02 22:00")&((calls_for_service['cfs-date'])<"2023-04-02 23:00")].sort_values('cfs-date')[['Incident No.', 'cfs-date','Description','Block Location','Call Source','Disposition','Case No']]

,Incident No.,cfs-date,Description,Block Location,Call Source,Disposition,Case No
58177,230402L00027628,2023-04-02 22:02:07.360,Petty Theft <=$950,700 BLOCK OTAY LAKES RD,citizen,ASSISTED,NaN
61750,230402L00027630,2023-04-02 22:10:11.057,Disturbance - General,1400 BLOCK HILLTOP DR,citizen,HANDLED NO REPORT,NaN
161548,230402L00027629,2023-04-02 22:11:25.747,Extra Patrol,400 BLOCK CALLE LA MARINA,officer,HANDLED NO REPORT,NaN
60203,230402L00027634,2023-04-02 22:16:57.020,[Protected Call Type],900 BLOCK BROADWAY,citizen,ASSISTED,NaN
83240,230402L00027635,2023-04-02 22:28:21.110,Disturbance - General,400 BLOCK THIRD AV,citizen,HANDLED NO REPORT,NaN
180735,230402L00027638,2023-04-02 22:35:46.957,Disturbance - General,1700 BLOCK E PALOMAR ST,citizen,HANDLED NO REPORT,NaN
96612,230402L00027639,2023-04-02 22:36:04.770,Vehicle Theft,100 BLOCK D ST,citizen,HANDLED NO REPORT,NaN
108158,230402L00027640,2023-04-02 22:37:36.563,Disturbance - Noise,200 BLOCK G ST,citizen,ASSISTED,NaN
156176,230402L00027641,2023-04-02 22:58:42.340,Suspicious Person,1700 BLOCK SOLSTICE AV,citizen,HANDLED NO REPORT,NaN


In [7]:
hbs = flight_blocks[flight_blocks['GEOID20']=='060730131032011']
hbs = hbs[hbs['seconds'].astype(int)>60]
hbs_flight_ids = hbs['id'].drop_duplicates().values
hbs_flights = cv_flights[cv_flights['id'].isin(hbs_flight_ids)]


In [8]:
hbs_flights[hbs_flights['date']=='2022-08-31']

,lat_map,lon_map,time,id,date,time_s,address_map,incident_id,type,department,drone-date
5328,32.6067906,-117.0800823,12:56pm,3569ebf8f25bb02d7d9ea734bea24004,2022-08-31,12:56pm,600 Oxford St,CVL072215,Subjects causing a disturbance,cvpd,2022-08-31
5329,32.6099128,-117.0816837,11:33am,a6ad86d47200eb4c8fb2348ac2b7e368,2022-08-31,11:33am,1100 Broadway,CVL072198,Subject causing a disturbance,cvpd,2022-08-31


In [9]:
calls_for_service[calls_for_service['Incident No.'].astype(str).str.contains('7995')].sort_values('cfs-date')

,Incident No.,Date,Description,Priority,Disposition,Call Source,Zip Code,Block Location,Case No,Address,...,Full FIPS (block),Full FIPS (tract),Metro/Micro Statistical Area Name,Metro/Micro Statistical Area Code,Metro/Micro Statistical Area Type,Combined Statistical Area Name,Combined Statistical Area Code,Metropolitan Division Area Name,Metropolitan Division Area Code,cfs-date
67881,210712L00057995,2021-07-12 22:17:05.923,Disturbance - Noise,4,ASSISTED,citizen,91911,1000 BLOCK BROADWAY,NaN,"1000 BLOCK BROADWAY, Chula Vista, CA, 91911",...,060730131032002,06073013103,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-07-12 22:17:05.923
43209,210815L00067995,2021-08-15 00:12:47.277,Drunk Driver,3,ASSISTED,citizen,91911,1100 BLOCK BROADWAY,NaN,"1100 BLOCK BROADWAY, Chula Vista, CA, 91911",...,060730131032011,06073013103,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-08-15 00:12:47.277
131916,210916L00077995,2021-09-16 13:03:05.310,Found,3,HANDLED NO REPORT,citizen,91913,1800 BLOCK MONACO DR,NaN,"1800 BLOCK MONACO DR, Chula Vista, CA, 91913",...,060730133181015,06073013318,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-09-16 13:03:05.310
119152,210922L00079950,2021-09-22 23:59:16.223,Minor Injury Collision,2,ASSISTED,citizen,91915,2800 BLOCK OLYMPIC PW,2112042.0,"2800 BLOCK OLYMPIC PW, Chula Vista, CA, 91915",...,060730133214002,06073013321,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-09-22 23:59:16.223
86936,210923L00079952,2021-09-23 00:02:11.980,Check a Person's Well Being,3,HANDLED NO REPORT,citizen,91911,200 BLOCK QUINTARD ST,NaN,"200 BLOCK QUINTARD ST, Chula Vista, CA, 91911",...,060730132033002,06073013203,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-09-23 00:02:11.980
46068,210923L00079955,2021-09-23 00:37:28.123,Disturbance - General,2,ASSISTED,citizen,91910,700 BLOCK BROADWAY,2112043.0,"700 BLOCK BROADWAY, Chula Vista, CA, 91910",...,060730126003000,06073012600,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-09-23 00:37:28.123
49996,210923L00079959,2021-09-23 01:17:47.650,Suspicious Person,3,HANDLED NO REPORT,citizen,91911,1100 BLOCK FOURTH AV,NaN,"1100 BLOCK FOURTH AV, Chula Vista, CA, 91911",...,060730131043000,06073013104,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-09-23 01:17:47.650
165996,210923L00079958,2021-09-23 01:18:46.793,Disturbance - General,2,HANDLED NO REPORT,citizen,91913,1300 BLOCK KECK RD,NaN,"1300 BLOCK KECK RD, Chula Vista, CA, 91913",...,060730133173042,06073013317,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-09-23 01:18:46.793
127717,211020L00087995,2021-10-20 05:41:34.433,Incident Evaluation,3,ASSISTED,citizen,91910,100 BLOCK FIFTH AV,NaN,"100 BLOCK FIFTH AV, Chula Vista, CA, 91910",...,060730124023000,06073012402,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-10-20 05:41:34.433
58846,211121L00097995,2021-11-21 17:26:12.263,Suspicious Circumstances,3,HANDLED NO REPORT,citizen,91911,1200 BLOCK INDUSTRIAL BL,NaN,"1200 BLOCK INDUSTRIAL BL, Chula Vista, CA, 91911",...,060730132051002,06073013205,"San Diego-Chula Vista-Carlsbad, CA",41740,metropolitan,NaN,NaN,NaN,NaN,2021-11-21 17:26:12.263


In [10]:
compiled_flights =[] 

for _, row in tqdm(hbs_flights[hbs_flights['address_map'].astype(str).str.contains('oxford',case=False)].iterrows()):
    
    d = gpd.read_file(f"/Volumes/easystore/Drones/flights/kml/{row['id']}.kml",driver='KML')
    d['drone-id'] = row['id']
    d['address_map'] = row['address_map']
    d['drone-date'] = row['drone-date']
    d['incident_id'] = row['incident_id']
    d['type']=row['type']
     
    compiled_flights.append(d)
    

0it [00:00, ?it/s]

DriverError: /Volumes/easystore/Drones/flights/kml/69fd8625c3bddc25dc1098bbe5afc0af.kml: No such file or directory

In [ ]:
paths = pd.concat(compiled_flights,ignore_index=True)
paths = paths[paths['Name']!="Home"]


In [ ]:
paths.to_csv('./hbs.csv',index=False)

In [ ]:
hbs_flights[hbs_flights['address_map'].astype(str).str.contains('oxford',case=False)]